In [1]:
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import yfinance as yf

In [2]:
ticker_symbols = [
    "AMD",
    "AAPL",
    "AMZN",
    "META",
    "MSFT",
    "NVDA",
    "PYPL",
    "TSLA",
    "TWTR",
]

In [3]:
ticker_price_df = {}
for ticker in ticker_symbols:
    price_df = yf.download(
        ticker,
        start="2020-01-01",
        end="2021-12-31",
    )
    if price_df.empty:
        continue
    price_df = price_df.reset_index()
    price_df.insert(0, "ticker", ticker)
    ticker_price_df[ticker] = price_df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
# Create a calendar
nyse = mcal.get_calendar('NYSE')
# Get NYSE trading dates in 2022
trading_dates_in_2022 = nyse.schedule(start_date='2022-01-01', end_date='2022-12-31').index.to_list()

In [5]:
simulated_price_dfs = []
for (ticker, price_df) in ticker_price_df.items():
    pct_change = price_df["Adj Close"].pct_change()
    pct_change_std = pct_change.std()
    
    num_of_simulation = 5
    ticker_price_list = []
    for simulation_id in range(num_of_simulation):
        # simulated stock prices for one year(2022)
        for (idx, dt) in enumerate(trading_dates_in_2022):
            if idx == 0:
                prev_price = price_df["Adj Close"].iat[-1]
            else:
                prev_price = ticker_price_list[-1]["price"]
                
            price = prev_price * (1 + np.random.normal(0, pct_change_std))
            ticker_price_list.append(
                {
                    "ticker": ticker,
                    "simulation_id": simulation_id,
                    "date": dt,
                    "price": price,
                }
            )
            
    simulated_price_dfs.append(pd.DataFrame(ticker_price_list))
    
simulated_price_df = pd.concat(simulated_price_dfs, ignore_index=True)

In [6]:
simulated_price_df

,ticker,simulation_id,date,price
0,AMD,0,2022-01-03,145.734634
1,AMD,0,2022-01-04,149.323729
2,AMD,0,2022-01-05,158.090569
3,AMD,0,2022-01-06,165.261684
4,AMD,0,2022-01-07,171.234328
...,...,...,...,...
11290,TWTR,4,2022-12-23,61.586612
11291,TWTR,4,2022-12-27,59.596896
11292,TWTR,4,2022-12-28,57.731234
11293,TWTR,4,2022-12-29,58.253338
